# Set-up and loading the data

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util, models
import os

In [ ]:
path_data = '..\\data_structured'

In [ ]:
df_comb = pd.read_pickle(os.path.join(path_data, 'comb.pkl'))

# Sentiment Analysis (ClimateBERT)

In [ ]:
# unifying the sentiment probabilities and labels into one score
def calculate_score(row):
    if (row['sentiment'] == 'negative')|(row['sentiment'] == 'risk'):
        return row['sentiment_probability']*(-1)
    elif row['sentiment'] == 'neutral':
        return row['sentiment_probability']*0
    else:
        return row['sentiment_probability']

In [ ]:
# creating a function to get sentiment scores and labels
def get_sentiment_scores(df, model):
    pipe_sentiment = pipeline(model = model, device = 0, batch_size = 64)
    sentences = df['sentence'].tolist()  # Convert the column to a list

    results = pipe_sentiment(sentences)
    df['sentiment'] = [result['label'] for result in results]
    df['sentiment_probability'] = [result['score'] for result in results]
    df['sentiment_score'] = df.apply(calculate_score, axis = 1)
    return df

In [ ]:
df_climate = df_comb.copy()

In [ ]:
df_climate = get_sentiment_scores(df_climate, 'climatebert/distilroberta-base-climate-sentiment')

In [ ]:
df_climate['sentiment'].value_counts()

In [ ]:
# this is to get the finbert scores for the comparison in the discussion section
# %%time
# df_comb = get_sentiment_scores(df_comb, 'ProsusAI/finbert')

In [ ]:
sust_topics = [0,1,2,3,4,5,6,7,8,10,11,20,25]
# this was a test to see whether excluding certain topics would change results significantly (specifically greenwashing accusations)
#lim_topics = [0,1,2,3,4,5,6,7,8,10,11,20]
# df_analyze = df_comb[df_comb['topics'].isin(sust_topics)]

In [ ]:
df_analyze = df_climate[df_climate['topics'].isin(sust_topics)]
df_analyze.reset_index(inplace = True, drop = True)
df_analyze.shape

### Score calculation

In [ ]:
def gw_score_sent_average(firm, cluster, df, mode= None):
    # calculate the average sentiment score for this firm in this cluster across its report
    rep_score = df[(df['company']==firm)&(df['topics']==cluster)&(df['doc_type']=='report')]['sentiment_score'].mean(skipna=True)
    # calculate the average sentiment score for this firm in this cluster across its news coverage
    news_score = df[(df['company']==firm)&(df['topics']==cluster)&(df['doc_type']=='news')]['sentiment_score'].mean(skipna=True)
    if mode == 'weighted':
        # for calculating the cluster importance as described by Boelders, we need to divide the number of cluster sentences for this company by its total number of sentences
        cluster_sentences = len(df[(df['company']==firm)&(df['topics']==cluster)])
        total_sentences = len(df[df['company']==firm])
        cl_importance = cluster_sentences/total_sentences
        return (rep_score-news_score)*cl_importance
    else:
        return (rep_score-news_score)

In [ ]:
# calculating the average sentiment discrepancy without clusters
gw_database_total = {}
for firm in set(df_analyze['company']):
     # calculate the average sentiment sc|ore for this firm for symbolic actions
    rep_score = df_analyze[(df_analyze['company']==firm)&(df_analyze['doc_type']=='report')]['sentiment_score'].mean(skipna=True)
    # calculate the average sentiment score for this firm for substantive actions
    news_score = df_analyze[(df_analyze['company']==firm)&(df_analyze['doc_type']=='news')]['sentiment_score'].mean(skipna=True)
    gw_database_total[firm] = (rep_score-news_score)

In [ ]:
# calculating with clusters
gw_database = {}
for cluster in list(set(df_analyze['topics'])):
    firm_cluster_score = {}
    for firm in set(df_analyze['company']):
        firm_cluster_score[firm] = gw_score_sent_average(firm, cluster, df_analyze)
    gw_database[cluster]= firm_cluster_score

In [ ]:
# the dataframe with the firm level scores
df_gw_total = pd.DataFrame(gw_database_total.items(),columns=['company', 'clim_sentiment_overall'])
df_gw_total

In [ ]:
# dataframe with topic level scores
df_gw = pd.DataFrame.from_dict(gw_database)
df_gw

In [ ]:
df_gw.reset_index(inplace = True)
df_gw.rename(columns = {'index':'company'}, inplace = True)

In [ ]:
# calculating the average across topics
df_gw['clim_sentiment_cluster_average'] = df_gw[sust_topics].mean(axis = 1)
df_gw

In [ ]:
# merging the two datasets together and exporting it
merged_df = df_gw_total.copy()
merged_df = merged_df.merge(df_gw[['company','clim_sentiment_cluster_average']])
# merged_df.to_csv('sentiment_scores.csv', index = False)

In [ ]:
lim_topics

In [ ]:
df_analyze = df_comb[df_comb['topics'].isin(sust_topics)]
df_analyze.reset_index(inplace = True, drop = True)
df_analyze.shape

In [ ]:
df_analyze = df_comb[df_comb['topics'].isin(lim_topics)]
df_analyze.reset_index(inplace = True, drop = True)
df_analyze.shape

In [ ]:
df_report = df_analyze[df_analyze['doc_type']=='report']
df_article = df_analyze[df_analyze['doc_type']=='news']
df_report.reset_index(drop = True, inplace = True)
df_article.reset_index(drop = True, inplace = True)

In [ ]:
df_similarity = df_report.copy()

In [ ]:
%%time
similarity_scores = []
for i,row in df_similarity.iterrows():
    # define our query (i.e. claim) and the company it's related to 
    query_embedding = row['embeddings']
    company = row['company']
    # search only the article embeddings/sentences of the specific company
    corpus_embeddings = df_article[(df_article['company']==company)]['embeddings'].values
    top_10 = util.semantic_search(torch.Tensor(query_embedding), torch.Tensor(np.array(list(corpus_embeddings))), top_k = 10)
    try:
        sim_score = pd.DataFrame(top_10[0])['score'].mean()
    except:
        sim_score = 'no sentences'
    similarity_scores.append(sim_score)

df_similarity['sentence_similarity'] = similarity_scores

In [ ]:
gw_sim_total = {}
for firm in set(df_analyze['company']):
    gw_sim_total[firm] = df_similarity[(df_similarity['company']==firm)&(df_similarity['sentence_similarity']!='no sentences')]['sentence_similarity'].mean(skipna=True)
gw_sim_total

In [ ]:
df_gw_sim_total = pd.DataFrame(gw_sim_total.items(),columns=['company', 'sim_score'])
df_gw_sim_total['sim_score'] = NormalizeData(df_gw_sim_total['sim_score'])
df_gw_sim_total['dissimilarity_total_lim'] = 1 - df_gw_sim_total['sim_score']

In [ ]:
df_gw_sim_total

In [ ]:
# df_similarity_complete = df_gw_sim_total[['company','dissimilarity_total']]
# df_similarity_complete
df_similarity_complete = pd.merge(df_similarity_complete, df_gw_sim_total[['company','dissimilarity_total_lim']])

In [ ]:
df_similarity_complete.to_csv('similarity_overall.csv', index = False)

In [ ]:
%%time

# cluster based sentence similarity

similarity_scores = []
for i,row in df_similarity.iterrows():
    # define our query (i.e. claim) and the company it's related to 
    query_embedding = row['embeddings']
    company = row['company']
    cluster = row['topics']
    # search only the article embeddings/sentences of the specific company
    corpus_embeddings = df_article[(df_article['company']==company)&(df_article['topics']==cluster)]['embeddings'].values
    top_10 = util.semantic_search(torch.Tensor(query_embedding), torch.Tensor(np.array(list(corpus_embeddings))), top_k = 10)
    try:
        sim_score = pd.DataFrame(top_10[0])['score'].mean()
    except:
        sim_score = 'no sentences'
    similarity_scores.append(sim_score)

df_similarity['sentence_similarity'] = similarity_scores

In [ ]:
df_similarity

In [ ]:
gw_database_sim = {}
for cluster in list(set(df_analyze['topics'])):
    firm_cluster_score = {}
    for firm in set(df_analyze['company']):
        # weighted average
        cluster_sentences = len(df_analyze[(df_analyze['company']==firm)&(df_analyze['topics']==cluster)])
        total_sentences = len(df_analyze[df_analyze['company']==firm])
        cl_importance = cluster_sentences/total_sentences
        firm_cluster_score[firm] = (df_similarity[(df_similarity['company']==firm)&(df_similarity['topics']==cluster)&(df_similarity['sentence_similarity']!='no sentences')]['sentence_similarity'].mean(skipna=True))*cl_importance
    gw_database_sim[cluster]= firm_cluster_score

In [ ]:
df_gw_sim = pd.DataFrame.from_dict(gw_database_sim)
df_gw_sim

In [ ]:
# run this for regular average
df_gw_sim['sim_average_cluster_lim'] = df_gw_sim.mean(axis = 1, skipna = True)
df_gw_sim['sim_average_cluster_lim'] = NormalizeData(df_gw_sim['sim_average_cluster_lim'])
df_gw_sim['dissimilarity_average_cluster_lim'] = 1 - df_gw_sim['sim_average_cluster_lim']
df_gw_sim

In [ ]:
# run this for weighted average
df_gw_sim['sim_waverage_cluster_lim'] = df_gw_sim[lim_topics].mean(axis = 1, skipna = True)
df_gw_sim['sim_wsum_cluster_lim'] = df_gw_sim[lim_topics].sum(axis = 1, skipna = True)
df_gw_sim['sim_waverage_cluster_lim'] = NormalizeData(df_gw_sim['sim_waverage_cluster_lim'])
df_gw_sim['sim_wsum_cluster_lim'] = NormalizeData(df_gw_sim['sim_wsum_cluster_lim'])
df_gw_sim['dissimilarity_waverage_cluster_lim'] = 1 - df_gw_sim['sim_waverage_cluster_lim']
df_gw_sim['dissimilarity_wsum_cluster_lim'] = 1 - df_gw_sim['sim_wsum_cluster_lim']
df_gw_sim


In [ ]:
df_gw_sim.reset_index(inplace = True)
df_gw_sim.rename(columns = {'index':'company'}, inplace = True)

In [ ]:
# average
df_similarity_complete = pd.merge(df_similarity_complete, df_gw_sim[['company','dissimilarity_average_cluster_lim']])

In [ ]:
# weighted average
df_similarity_complete = pd.merge(df_similarity_complete, df_gw_sim[['company','dissimilarity_waverage_cluster_lim', 'dissimilarity_wsum_cluster_lim']])

In [ ]:
df_similarity_complete.to_csv('similarity_overall.csv', index = False)

In [ ]:
df_similarity_complete